In [ ]:
from __future__ import print_function
import keras
import numpy as np
from keras.models import Sequential
from keras.models import Model

from keras import optimizers, applications, regularizers

from keras.applications import Xception
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator


from keras.layers import Dense, Dropout, Activation, Flatten,core
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D,Conv1D, MaxPooling2D
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Input
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)

In [ ]:
img_width, img_height = 299,299
train_data_dir = 'train/'
validation_data_dir = 'validation/'
test_data_dir = 'test/'
nb_train_samples = 7105 
nb_validation_samples =  2368
epochs = 600 
batch_size = 4
num_classes = 41

In [ ]:
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(img_width, img_height, 3))  # this assumes K.image_data_format() == 'channels_last'

#model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)
base_model = applications.Xception(input_tensor=input_tensor,pooling = 'avg', include_top=False,weights=None)

from keras.layers import BatchNormalization, Dropout
# add a global spatial average pooling layer
x = base_model.output

#x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dropout(0.8)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range = 0.8,
        horizontal_flip = True,
        fill_mode = 'reflect'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width,img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = True)

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width,img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False)


In [ ]:
from keras.optimizers import SGD
sgd = SGD(lr=0.004, momentum=0.9,decay = 1e-6, nesterov = True)

#model.compile(optimizer='adadelta', loss='categorical_hinge', metrics=['accuracy'])
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
checkpoint = keras.callbacks.ModelCheckpoint('logs/' + 'weights-{epoch:02d}-{val_acc:.2f}-{val_loss:.2f}.h5',
                                monitor='val_loss',save_best_only=True, save_weights_only=True, verbose=1)
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=[checkpoint,early])

In [ ]:
model.load_weights('logs/weights-22-0.66-1.28.h5')

In [ ]:
test_generator = test_datagen.flow_from_directory(
        'test/',
        target_size=(img_width,img_height),
        batch_size=4,
        class_mode=None,
        shuffle = False)

In [ ]:
val_results = model.predict_generator(
                             validation_generator, 
                             steps = 2368 // batch_size)
val_eval = model.evaluate_generator(
                             validation_generator, 
                             steps = 2368 // batch_size)
print(val_eval)

In [ ]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
test_results = model.predict_generator(
                             test_generator, 
                             steps = 9400 // batch_size,verbose = 1)

In [ ]:
predictions = [list(np.argsort(test_results[i])[::-1][:3]) for i in range(len(test_results))]

In [ ]:
def preds_to_labels(p, labels):
    predictions = [list(np.argsort(p[i])[::-1][:3]) for i in range(len(p))]
    prediction_labels = []
    
    for pred in predictions:
        label_list = []
        for output in pred:
            label_list.append(labels[output])
        prediction_labels.append(label_list)
    return prediction_labels

In [ ]:
indi = train_generator.class_indices
labels = [' ']*41
for e in indi:
    labels[indi[e]] = e
    
predi = preds_to_labels(test_results,labels)

In [ ]:
import pandas as pd
def create_submission(predictions, name='submission.csv'):
    predictions = ['{} {} {}'.format(x[0], x[1], x[2]) for x in predictions]
    submission = pd.read_csv('../freesound_lgbm/input/sample_submission.csv')
    submission.label = predictions
    submission.to_csv('{}'.format(name), index=False)
    print("Submission saved to '{}'".format(name))

In [ ]:
create_submission(predi,name='tryout.csv')

In [ ]:
predictions = [list(np.argsort(test_results[i])[::-1][:3]) for i in range(len(test_results))]
actual = [[i] for i in y_valid]
valid_score = mapk(actual, predictions, k=3)

print(valid_score)